<a href="https://colab.research.google.com/github/sinjy1203/deep-learning/blob/master/hands_on_deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import tensorflow as tf
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [0]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [0]:
sess.close()

In [0]:
with tf.Session() as sess:
  x.initializer.run()
  y.initializer.run()
  result = f.eval()
  print(result)

42


In [0]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
  init.run()
  result = f.eval()

In [0]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [0]:
graph = tf.Graph()
with graph.as_default():
  x2 = tf.Variable(2)
  print(x2.graph is tf.get_default_graph())

True


In [0]:
x2.graph is tf.get_default_graph()

False

In [0]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
with tf.Session() as sess:
  print(y.eval())
  print(z.eval())

10
15


In [0]:
import numpy as np
from sklearn.datasets import fetch_california_housing
import tensorflow as tf
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
x = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='x')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
xt = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(xt, x)), xt), y)

with tf.Session() as sess:
  theta_value = theta.eval()
  print(theta_value)

[[-3.67372932e+01]
 [ 4.37366009e-01]
 [ 9.47520509e-03]
 [-1.08159676e-01]
 [ 6.48537397e-01]
 [-3.84734449e-06]
 [-3.79239232e-03]
 [-4.19136107e-01]
 [-4.32144403e-01]]


In [0]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
housing_data_scale = std_scaler.fit_transform(housing.data)
housing_data_scale_bias = np.c_[np.ones((m, 1)), housing_data_scale]
n_epochs = 1000
learning_rate = 0.01
x = tf.constant(housing_data_scale_bias, dtype=tf.float32, name='x')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(x, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init)
  for epoch in range(n_epochs):
    if epoch % 100 == 0:
      print("epoch", epoch, "mse = ", mse.eval())
    sess.run(training_op)
  best_theta = theta.eval()

epoch 0 mse =  9.607067
epoch 100 mse =  0.5340985
epoch 200 mse =  0.52520466
epoch 300 mse =  0.5244316
epoch 400 mse =  0.5243356
epoch 500 mse =  0.52432287
epoch 600 mse =  0.52432126
epoch 700 mse =  0.524321
epoch 800 mse =  0.524321
epoch 900 mse =  0.52432096


In [0]:
a = tf.placeholder(tf.float32, shape=(None, 3))
b = a + 5
with tf.Session() as sess:
  b_val_1 = b.eval(feed_dict={a: [[1, 2, 3]]})
  b_val_2 = b.eval(feed_dict={a: [[4, 5, 6], [7, 8, 9]]})
print(b_val_1)
print(b_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [0]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
housing_data_scale = std_scaler.fit_transform(housing.data)
housing_data_scale_bias = np.c_[np.ones((m, 1)), housing_data_scale]
n_epochs = 10
learning_rate = 0.01

x = tf.placeholder(dtype=tf.float32, shape=(None, n + 1), name="x")
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(x, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

batch_size = 100
n_batchs = int(np.ceil(m / batch_size))
def fetch_batch(epoch, batch_index, batch_size):
  np.random.seed(epoch * n_batchs + batch_index)
  indices = np.random.randint(m, size=batch_size)
  x_batch = housing_data_scale_bias[indices]
  y_batch = housing.target.reshape(-1, 1)[indices]
  return x_batch, y_batch

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
  #saver.restore(sess, "/content/gdrive/My Drive/tmp/my_model.ckpt")
  sess.run(init)
  for epoch in range(n_epochs):
    for batch_index in range(n_batchs):
      x_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
      _, cost = sess.run([training_op, mse], feed_dict={x: x_batch, y: y_batch})
    save_path = saver.save(sess, "/content/gdrive/My Drive/tmp/my_model.ckpt")
    print(cost)
  best_theta = theta.eval()
  print(best_theta)

0.54261464
0.61449236
0.5230496
0.6708452
0.40450543
0.50794595
0.3609971
0.39653167
0.69455475
0.37981114
[[ 2.069967  ]
 [ 0.82023007]
 [ 0.11956575]
 [-0.22237983]
 [ 0.30452585]
 [ 0.00445741]
 [-0.01205201]
 [-0.8988268 ]
 [-0.86161935]]


In [0]:
with tf.Session() as sess:
  saver.restore(sess, "/content/gdrive/My Drive/tmp/my_model.ckpt")
  best_theta_restore = theta.eval()

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/tmp/my_model.ckpt


In [0]:
np.allclose(best_theta, best_theta_restore)

True

In [0]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "/content/gdrive/My Drive/{}/run-{}".format(root_logdir, now)

In [0]:
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

n_epochs = 1000
learning_rate = 0.01 
x = tf.placeholder(dtype=tf.float32, shape=(None, n + 1), name='x')
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, -1, seed=42), name='theta')
y_pred = tf.matmul(x, theta, name='y_predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph)

n_epochs = 10
batch_size = 100
n_batches = int(np.celi(m / batch_size))
with tf.Session() as sess:
  sess.run(init)
  for epoch in range(n_epochs):
    for batch_index in range(n_batches):
      x_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
      if batch_index % 10 == 0:
        summary_str = mse_summary.eval(feed_dict={x: x_batch, y: y_batch})
        step = epoch * n_batches + batch_index
        file_writer.add_summary(summary_str, step)
      sess.run(training_op, feed_dict={x: x_batch, y: y_batch})
  best_theta = theta.eval()
file_writer.close()

TypeError: ignored

In [0]:
import tensorflow as tf
import numpy as np
from tensorboardcolab import *
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "/content/gdrive/My Drive/{}/run-{}/".format(root_logdir, now)

Using TensorFlow backend.


In [0]:
n_epochs = 1000
learning_rate = 0.01
tbc = TensorBoardColab()

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

Wait for 8 seconds...
TensorBoard link:
https://ffc539c4.ngrok.io


In [0]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tbc.get_writer()
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [0]:

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [0]:
with tf.Session() as sess:                                                        # 책에는 없습니다.
    sess.run(init)                                                                # 책에는 없습니다.

    for epoch in range(n_epochs):                                                 # 책에는 없습니다.
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval() 

In [0]:
file_writer.close()

In [0]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

n_features = 3
x = tf.placeholder(dtype=tf.float32, shape=(None, n_features), name='x')
w1 = tf.Variable(tf.random_normal((n_features, 1)), name='w1')
w2 = tf.Variable(tf.random_normal((n_features, 1)), name='w2')
b1 = tf.Variable(0.0, name='b1')
b2 = tf.Variable(0.0, name='b2')
z1 = tf.add(tf.matmul(x, w1), b1, name='z1')
z2 = tf.add(tf.matmul(x, w2), b2, name='z2')
relu1 = tf.maximum(z1, 0, name='relu1')
relu2 = tf.maximum(z2, 0, name='relu2')

output = tf.add(relu1, relu2, name='output')

In [0]:
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

def relu(x):
  w_shape = (int(x.get_shape()[1]), 1)
  w = tf.Variable(tf.random_normal(w_shape), name='w')
  b = tf.Variable(0.0, name='b')
  z = tf.add(tf.matmul(x, w), b, name='z')
  return tf.maximum(z, 0, name='relu')

n_features = 3
x = tf.placeholder(dtype=tf.float32, shape=(None, n_features), name='x')
relus = [relu(x) for i in range(5)]
output = tf.add_n(relus, name='output')

In [0]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

def relu(x):
  threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
  w_shape = (int(x.get_shape()[1]), 1)
  w = tf.Variable(tf.random_normal(w_shape), name='w')
  b = tf.Variable(0.0, name='b')
  z = tf.add(tf.matmul(x, w), b, name='z')
  return tf.maximum(z, threshold, name='max')
n_features = 3
x = tf.placeholder(dtype=tf.float32, shape=(None, n_features), name='x')
relus = []
for relu_index in range(5):
  with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
    relus.append(relu(x))
output = tf.add_n(relus, name="output")